In [1]:
import json
import os
import requests
import time
from tqdm import tqdm


In [2]:
filename = 'data.json'
with open(filename, 'r') as f:
  text = f.read()
data = json.loads(text)

In [3]:
data[0]# a.b.c  a#b#c

{'_id': {'$oid': '5d0dd038082f6d0c23bd105c'},
 'dict_content': {'SJTU-al-mmc-property': {'information': {'upload-date': '2019-06-22',
    'data-source': 'SJTU',
    'doi': None},
   '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
   'mechanical-property': {'tensile-strength': {'unit': 'MPa', 'value': 450},
    'percentage-elongation-at-maximum-force': {'unit': '%', 'value': 1},
    'specifies-the-plastic-elongation-strength': {'specifies-the-plastic-elongation-strength-record': {'strength': {'unit': 'MPa',
       'value': 449},
      'rp': 0.2}},
    'tensile-curve': {'file': {'url': '/mdcs-sjtu/rest/blob/download/5d0f3348082f6d0c23bd1fcf/',
      'name': '1.xls'}},
    'youngs-modulus': {'unit': 'GPa', 'value': 81.1},
    'number-of-test-sample': 1,
    'volume-ratio-of-reinforcement-in-test-sample': {'unit': '%', 'value': 15},
    'break-strength': {'unit': 'MPa', 'value': 445},
    'test-date': '2019-04-10'}}},
 'title': 'SiC/Al tensile test',
 'last_modification_date': {

In [4]:
# 处理附件
# 所有的文件放在files/文件夹下，为了避免重名，每个数据 新建一个 名为数据ID的文件夹，附件放入其中
if not os.path.exists('files'):
    os.makedirs('files')
with open('files/NA.xls', 'w+'):
  pass
def download_file(url, path, filename):
  r = requests.get(url).content
  with open(os.path.join(path, filename), 'wb+') as f:
    f.write(r)

In [5]:
# 进度条
pbar = tqdm(total=len(data))


converted_data = []

for idx, d in enumerate(data):
  # meta data
  title = d.get('title')
  
  # SJTU-al-mmc-property
  # information
  p = d['dict_content'].get('SJTU-al-mmc-property', {})
  upload_date = p.get('information', {}).get('upload-date')
  data_source = p.get('information', {}).get('data-source')
  # mechanical-property
  m = p.get('mechanical-property', {})
  tensile_strength = m.get('tensile-strength', {}).get('value')
  percentage_elongation_at_maximum_force = m.get('percentage-elongation-at-maximum-force', {}).get('value')
  
  strength = m.get('specifies-the-plastic-elongation-strength', {}).get('specifies-the-plastic-elongation-strength-record', {}).get('strength', {}).get('value')
  rp = m.get('specifies-the-plastic-elongation-strength', {}).get('specifies-the-plastic-elongation-strength-record', {}).get('rp')
  tensile_curve = m.get('tensile-curve', {}).get('file', {}).get('name', '')
  youngs_modulus = m.get('youngs-modulus', {}).get('value')
  number_of_test_sample = m.get('number-of-test-sample')
  volume_ratio_of_reinforcement_in_test_sample = m.get('volume-ratio-of-reinforcement-in-test-sample', {}).get('value')
  break_strength = m.get('break-strength', {}).get('value')
  test_date = m.get('test-date')
  tester = m.get('tester')
  
  # 下载文件
  path = os.path.join('files', str(idx+1))
  filename = m.get('tensile-curve', {}).get('file', {}).get('name', '')

  if filename:
    # 如果文件名以xls结尾，说明可能可以下载
    if tensile_curve.endswith('xls'):
        file_path = os.path.join('files/', str(idx+1), tensile_curve)
    # 不重复下载，如果不存在此文件，则下载
    if not os.path.exists(os.path.join(path, filename)):
      if not os.path.exists(path):
        os.makedirs(path)
      try:
        download_url = 'http://matdata.shu.edu.cn' + m.get('tensile-curve', {}).get('file', {}).get('url', '')
        download_file(download_url, path, filename)
      except Exception as e:
        # 下载失败
        print('下载文件{}出错，错误信息：{}'.format(filename, str(e)))
        
        file_path = 'files/NA.xls'
  else:
    file_path = 'files/NA.xls'
    
  data_template = {
            "meta": {
                "数据 ID": title,
                "标题": title,
                "DOI": "",
                "数据摘要": "铝基复合材料 性能数据",
                "关键词": "铝基复合材料 性能数据",
                "来源": "MGE-SOURCE_HEADER v1 0100 10 #",
                "引用": "",
                "其他信息": "project: 2018YFB0704400；subject: 2018YFB0704401",
                "数据生产机构": "SJTU",
                "数据生产者": "SJTU",
                "公开时间": "0",
                "公开范围": ""
            },
            "content": {
                "力学性能": {
                    "测试样品编号": number_of_test_sample,
                    "测试样品含量": volume_ratio_of_reinforcement_in_test_sample,
                    "测试日期": test_date,
                    "测试人": tester,
                    "弹性模量": youngs_modulus,
                    "规定塑性延伸强度": [
                        {
                            "规定非比例延伸强度": rp,
                            "强度": strength,
                        },
                    ],
                    "抗拉强度": tensile_strength,
                    "断裂强度": break_strength,
                    "最大力总伸长率": percentage_elongation_at_maximum_force,
                    "拉伸曲线": [
                        file_path,
                    ]
                }
            }
        }
  converted_data.append(data_template)
  pbar.update(1)
pbar.close()

  6%|▋         | 116/1825 [00:00<00:02, 755.31it/s]

下载文件114.xls出错，错误信息：HTTPConnectionPool(host='matdata.shu.edu.cn114.xls', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd2ef136550>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
下载文件116.xls出错，错误信息：HTTPConnectionPool(host='matdata.shu.edu.cn116.xls', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd2ef136710>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
下载文件117.xls出错，错误信息：HTTPConnectionPool(host='matdata.shu.edu.cn117.xls', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd2ef136630>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))


100%|██████████| 1825/1825 [00:00<00:00, 5496.27it/s]

下载文件24.xls出错，错误信息：HTTPConnectionPool(host='matdata.shu.edu.cn24.xls', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd2ef136860>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))
下载文件115.xls出错，错误信息：HTTPConnectionPool(host='matdata.shu.edu.cn115.xls', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fd2ef136828>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))


In [6]:
converted_data[20]

{'meta': {'数据 ID': 21,
  '标题': 'XF600M-2',
  'DOI': '',
  '数据摘要': '铝基复合材料 性能数据',
  '关键词': '铝基复合材料 性能数据',
  '来源': 'MGE-SOURCE_HEADER v1 0100 10 #',
  '引用': '',
  '其他信息': 'project: 2018YFB0704400；subject: 2018YFB0704401',
  '数据生产机构': 'SJTU',
  '数据生产者': 'SJTU',
  '公开时间': '0',
  '公开范围': 'public'},
 'content': {'力学性能': {'测试样品编号': 21,
   '测试样品含量': 15,
   '测试日期': '2018-11-17',
   '测试人': 'MY',
   '弹性模量': 89.3,
   '规定塑性延伸强度': [{'规定非比例延伸强度': 0.2, '强度': 366}],
   '抗拉强度': 415,
   '断裂强度': 406,
   '最大力总伸长率': 1.7,
   '拉伸曲线': ['files/21/21.xls']}}}

In [7]:
# 模板
with open('性能数据.json', 'r') as f:
  file_template = json.load(f)

file_template['data'] = converted_data

In [8]:
with open('final.json', 'w+') as f:
  json.dump(file_template, f, ensure_ascii=False)

In [9]:
a = "dict_content.xingneng"

In [13]:
fileds = a.split('.')

In [ ]:
for f in fields:
  j = j.get(f)